In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import random
from matplotlib import image
from sklearn.model_selection import train_test_split
import pickle
from sklearn.utils.class_weight import compute_class_weight
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from prettytable import PrettyTable 
from contextlib import redirect_stdout
from sklearn.metrics import auc
import time

In [2]:
def data_analysis_histogram(dir_path):
    """ Analisa o conjunto de dados

    Retorna o tamanho da entrada e a distribuição das classes 

    Args:
        dir_path: Caminho do conjunto de dados
        
    Returns:
        Distribuição das classes
    """
    
    class_dist = []
    for c in CLASSES:
        class_path = os.path.join(dir_path, c)
        class_dist.append(len(os.listdir(class_path)))

    # Apresenta um histograma da distribuição das classes
    if VERBOSE['ANALYSIS']:
        plt.figure(figsize=(16, 8))
        plt.title("Distribuição das classes")
        plt.barh(CLASSES, class_dist)
        for index, value in enumerate(class_dist):
            plt.text(value, index, str(value))
        plt.show()
    return class_dist


def data_analysis_image_size(dir_path):
    """ Analisa o tamanho da entrada

    Retorna o tamanho da entrada

    Args:
        dir_path: Caminho do conjunto de dados

    Returns:
        Dimensões da entrada
    """

    random.seed(SEED)
    random_class_path = os.path.join(dir_path, random.choice(CLASSES))
    random_img_name = random.choice(os.listdir(random_class_path))
    random_img_path = os.path.join(random_class_path, random_img_name)
    img = image.imread(random_img_path)

    # Apresenta aleatoriamente um exemplo do conjunto de dados e suas dimensões
    # Espera-se que todo o conjunto de dados possua a mesma dimensão
    if VERBOSE['ANALYSIS']:
        plt.figure(figsize=(16, 8))
        plt.title("%s - Altura: %d px x Largura: %d px" %
                  (random_img_path, img.shape[0], img.shape[1]))
        plt.imshow(img)

    return (img.shape[0], img.shape[1], 1)


def analyse_dataset(dir_path):
    """ Analisa o conjunto de dados

    Retorna o tamanho da entrada e a distribuição das classes.

    Args:
        dir_path: Caminho do conjunto de dados

    Returns:
        Dimensões da entrada,
        Distribuição das classes
    """

    class_dist = data_analysis_histogram(dir_path)
    input_shape = data_analysis_image_size(dir_path)
    return input_shape, class_dist


In [3]:
def load_dataset(dir_path):
    """ Carrega o conjunto de dados

    Carrega e retorna as entradas e saídas do conjunto de dados.

    Args:
        dir_path: Caminho do conjunto de dados
        
    Returns:
        Entrada do conjunto de dados,
        Saída do conjunto de dados
    """

    img_array = []
    class_array = []
    for c in CLASSES:
        class_path = os.path.join(dir_path, c)
        imgs_name = os.listdir(class_path)

        # Aleatoriamente carregar a parcela do conjunto de dados definida por DATASET_PERCENTAGE
        if DATASET_PERCENTAGE < 1:
            imgs_name = random.sample(imgs_name, k=int(
                len(imgs_name)*DATASET_PERCENTAGE))

        for i in imgs_name:
            img_array.append(image.imread(os.path.join(class_path, i)))
            class_array.append(c)
    if VERBOSE['LOADING']:
        print("Loaded %d images" % len(img_array))

    return np.array(img_array), np.array(class_array)


In [4]:
def split_dataset(x, y):
    """ Divide o conjunto de dados em treinamento e validação

    Divide a entrada e a saída em um conjunto de treinamento e de validação

    Args:
        x: entrada
        y: saída
        
    Returns:
        Entrada do conjunto de treinamento,
        Entrada do conjunto de validação,
        Saída do conjunto de treinamento,
        Saída do conjunto de validação
    """
    
    if VALIDATION_PERCENTAGE <= 0:
        return x, None, y, None

    # Porcetagem da divisão dada por VALIDATION_PERCENTAGE
    x_train, x_val, y_train, y_val = train_test_split(x,  y, test_size=VALIDATION_PERCENTAGE, random_state=SEED)
    if VERBOSE['SPLITTING']:
        print("Train size: %d\nValidation size: %d" % (len(x_train), len(x_val)))
    return x_train, x_val, y_train, y_val

In [5]:
def prepare_dataset_channel_position(x, input_shape):
    """ Reordenada as dimensões da entrada
    
    Reordenada a dimensão 'canal' conforme o formato esperado pelo classificador, podendo ser a primeira ou a última dimensão da matriz.
    Portanto, uma entrada do tipo (linha, coluna, canal) pode ser reordenada para (canal, linha, coluna) caso o formato exija que o canal venha antes.
    Args:
        x: entrada
        input_shape: dimensões da entrada
        
    Returns:
        Entrada preparada para o classificador
        Dimensões da entrada preparada para o classificador
    """

    img_lin,img_col,n_channels = input_shape
    if keras.backend.image_data_format() == 'channels_first':
        x = x.reshape(x.shape[0], n_channels, img_lin, img_col)
        input_shape = (n_channels, img_lin, img_col)
    else:
        x = x.reshape(x.shape[0], img_lin, img_col, n_channels)
        input_shape = (img_lin, img_col, n_channels)
    return x, input_shape

def prepare_dataset_input(x, input_shape):
    """ Prepara a entrada para o classificador
    
    A entrada é normalizada e suas dimensões reordenadas.

    Args:
        x: entrada
        input_shape: dimensões da entrada
        
    Returns:
        Entrada preparada para o classificador,
        Dimensões da entrada preparada para o classificador
    """

    x_scaled = x.astype('float32') / 255.0
    return prepare_dataset_channel_position(x_scaled, input_shape)

def prepare_dataset_output(y):
    """ Prepara a saída conjunto de dados para o classificador
    
    A saída é transformada em um vetor one-hot encoding.

    Args:
        y: saída
        
    Returns:
        Saída transformada
    """

    class_map = {x: i for i,x in enumerate(CLASSES)}
    y_code = [class_map[word] for word in y]
    y_categorical = keras.utils.to_categorical(y_code, len(CLASSES))
    return y_categorical

def prepare_dataset(x , y, input_shape):
    """ Prepara o conjunto de dados para o classificador
    
    Transforma a entrada e saída para alimentar o classificador.
    A entrada é normalizada e suas dimensões reordenadas.
    A saída é transformada em um vetor one-hot encoding.

    Args:
        x: entrada
        y: saída
        input_shape: dimensões da entrada
        
    Returns:
        Entrada preparada para o classificador,
        Saida preparada para o classificador,
        Dimensões da entrada preparada para o classificador
    """

    if x is None:
        return None, None, None
    x_scaled, input_shape = prepare_dataset_input(x, input_shape)
    y_categorical = prepare_dataset_output(y)
    return x_scaled , y_categorical, input_shape

In [6]:
def save_model(model, training_param, history, elapsed_minutes):
    """ Salva o modelo do classificador
    
    Salva o modelo, o resultados da execução do treinamento e o tempo de execução.

    Args:
        model: modelo do classificador
        training_param: função que retorna parâmetros de treinamento
        history: histórico do treinamento (saída do treinamento do classificador)
        elapsed_minutes: tempo de execução do treinamento
        
    Returns:
        Não há
    """

    result_directory = os.path.join(RESULT_PATH)

    # Cria diretório, caso não exista
    if not os.path.exists(result_directory):
        os.makedirs(result_directory)

    result_directory = os.path.join(result_directory,model.name)

    # Não permite a sobreescrita dos resultados
    if not os.path.exists(result_directory):
        os.makedirs(result_directory)
    else:
        raise ValueError("File already exists.")
    
    model_path = os.path.join(result_directory,'model')
    # Salva o modelo na pasta "model"
    model.save(model_path)

    execution_path = os.path.join(result_directory,'execution')

    # Salva os resultados da execução no arquivo "execution"
    optimizer, batch_size, epochs = training_param()

    execution = {
            'optimizer' : {'name': optimizer._name, 'hyper': optimizer._hyper},
            'batch_size': batch_size,
            'epochs': history.params['epochs'],
            'history': history.history, # Contém resultados das métricas conforme a progressão do treinamento
            'elapsed_minutes': elapsed_minutes
    }

    with open(execution_path, 'wb') as f:
        pickle.dump(execution, f)
    
    # Salva o modelo no arquivo "model_summary_txt" em um formato de leitura mais fácil.
    # Neste arquivo também é armazenado o tempo de execução do treinamento.
    pretty_model_path = os.path.join(result_directory, 'model_summary.txt')

    with open(pretty_model_path,'w') as f:
        f.write('Elapsed time: '+str(elapsed_minutes)+' min\n')
        with redirect_stdout(f):
            model.summary()

In [7]:
def load_model(model_name):
    """ Carrega o modelo do classificador
    
    Carrega o modelo do classificador armazenado no caminho dado.

    Args:
        model_name: nome do diretório que contém o modelo a ser carregado
        
    Returns:
        Modelo do classificador, histórico da execução do treinamento
    """
    
    result_directory = os.path.join(RESULT_PATH, model_name)
    if not os.path.exists(result_directory):
        raise ValueError("Folder not found.")
    
    model_path = os.path.join(result_directory,'model')
    model = keras.models.load_model(model_path)

    execution_path = os.path.join(result_directory,'execution')
    execution = pickle.load(open(execution_path, "rb"))
    
    return model, execution

In [8]:
def save_evaluation(model_name, evalution_name, all_scores, table):
    """ Salva o resultado de uma avaliação do classificador
    
    Salva o resultado de uma avaliação do classificador, tanto em um formato JSON como em uma tabela para facilitar a leitura.

    Args:
        model_name: nome do diretório do modelo em que a avaliação será salva
        evalution_name: nome da avaliação
        all_scores: resultado da avaliação
        table: tabela do resultado da avaliação
        
    Returns:
        Não há
    """
    
    result_directory = os.path.join(RESULT_PATH,model_name)
    if not os.path.exists(result_directory):
        raise ValueError("Folder not found.")
    
    # O resultado é salvo em um arquivo com o nome da avaliação e o sufixo "_score" no formato .json.
    score_path = os.path.join(result_directory, evalution_name + '_score.json')
    with open(score_path, 'w') as f:
        json.dump(all_scores, f, ensure_ascii=False, indent=4)

    # A tabela é salva em um arquivo com o nome da avaliação e o sufixo "_score_summary" no formato .txt.
    pretty_score_path = os.path.join(result_directory, evalution_name + '_score_summary.txt')

    with open(pretty_score_path,'w') as f:
        f.write(model_name+'\n')
        f.write(table.get_string())

In [9]:
def plot_evaluation_roc(evaluation_name, model, x, y):
    """ Desenha a curva ROC
    
    Desenha a curva ROC do modelo classificador com base na entrada e saída.

    Args:
        evalution_name: nome da avaliação
        model: modelo do classificador
        x: entrada
        y: saída
        
    Returns:
        Falso positivo,
        Verdadeiro positivo,
        Área sob a curva ROC
    """

    y_pred = model.predict(x)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Calcula a curva ROC para cada classe, baseado na predição do modelo e a saída real
    for i in range(len(CLASSES)):
        fpr[i], tpr[i], _ = roc_curve(y[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    plt.figure(figsize=(16, 16))
    for i in range(len(CLASSES)):
        plt.plot(
            fpr[i],
            tpr[i],
            label="ROC curve of {0} (AUC = {1:0.2f})".format(CLASSES[i], roc_auc[i]))
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc="lower right")

    # Salva a imagem com o nome da avaliação e o com o sufixo "_roc_curve"
    result_directory = os.path.join(RESULT_PATH, model.name)
    image_path = os.path.join(result_directory,evaluation_name+'_roc_curve.png')
    plt.savefig(image_path)

    # Exibe a curva ROC para cada classe
    if VERBOSE['EVALUATION']:
        plt.show()
    else:
        plt.close()
        
    return fpr, tpr, roc_auc
    
def evaluate_model_by_class(model, x, y):
    """ Avalia o modelo por classes
    
    Faz a avaliação do modelo para cada classe do problema.

    Args:
        model: modelo do classificador
        x: entrada
        y: saída
        
    Returns:
        Métrica por classe
    """
    
    def separate_by_class(x, y):
        """ Separa a entrada e saída por classe
    
        Função interna que separa a entrada e saída por classe

        Args:
            x: entrada
            y: saída
            
        Returns:
            Entrada para cada classe,
            Saída para cada classe
        """

        n_classes = y.shape[1]
        x_classified = [[] for _ in range(n_classes)]
        y_classified = [[] for _ in range(n_classes)]
        
        # Separa as classes
        for i,img in enumerate(y):
            index = np.where(img==1)[0][0]
            x_classified[index].append(x[i])
            y_classified[index].append(y[i])

        # Converte a entrada e saída para um array numpy
        for i in range(n_classes):
            x_classified[i] = np.array(x_classified[i])
            y_classified[i] = np.array(y_classified[i])
            
        return np.array(x_classified,dtype=object), np.array(y_classified,dtype=object)

    x_by_class, y_by_class = separate_by_class(x,y)
    
    score_by_class = []

    # Avalia o modelo para cada classe
    for x,y in zip(x_by_class,y_by_class):
        score = model.evaluate(x, y, verbose = 1 if VERBOSE['EVALUATION'] else 0)
        score_by_class.append(score)

    return score_by_class

def evaluate_model_confusion_matrix(evaluation_name, model, x, y):
    """ Calcula a matriz de confusão
    
    Faz o cálculo da matriz de confusão do modelo de classificação.

    Args:
        evaluation_name: nome da avaliação
        model: modelo do classificador
        x: entrada
        y: saída
        
    Returns:
        Matriz de confusão
    """

    def undoOneHotEncoding(y):
        """ Desfaz a matriz one-hot encoding
    
        Transforma a matriz one-hot encoding para sua forma original, com o nome das classes.

        Args:
            y: saída
            
        Returns:
            Saída transformada
        """

        return [CLASSES[i] for i in np.argmax(y, axis = 1)]
    
    y_pred = model.predict(x)
    y_pred_int = undoOneHotEncoding(y_pred)
    y_int = undoOneHotEncoding(y)

    # Cria a matriz de confusão baseado na previsão do modelo de classificação e o resultado real
    cm = confusion_matrix(y_int, y_pred_int)

    cm_df = pd.DataFrame(cm,
                        index = [CLASSES[i] for i in range(len(CLASSES))], 
                        columns = [CLASSES[i] for i in range(len(CLASSES))])
    plt.figure(figsize=(16,16))
    sns.heatmap(cm_df, annot=True)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual Values')
    plt.xlabel('Predicted Values')
    
    # Salva a imagem com o nome da avaliação e o com o sufixo "_confusion_matrix"
    results_directory = os.path.join(RESULT_PATH, model.name)
    image_path = os.path.join(results_directory,evaluation_name+'_confusion_matrix.png')
    plt.savefig(image_path)

    # Exibe a matriz de confusão
    if VERBOSE['EVALUATION']:
        plt.show()
    else:
        plt.close()
    return cm

def evaluate_model(evaluation_name, model, x, y):
    """ Avalia o modelo
    
    Faz a análise do modelo, fazendo a avalição por classe, gerando a matriz de confusão e desenhando a curva ROC.

    Args:
        evaluation_name: nome da avaliação
        model: modelo do classificador
        x: entrada
        y: saída
        
    Returns:
        Resultado da avaliação
    """

    if x is None:
        return None

    score = model.evaluate(x, y, verbose = 1 if VERBOSE['EVALUATION'] else 0)
    score_by_class = evaluate_model_by_class(model, x, y)
    cm = evaluate_model_confusion_matrix(evaluation_name, model,x, y)
    _, _, roc_auc = plot_evaluation_roc(evaluation_name, model, x, y)
    table = PrettyTable()
    table.add_column("Metrics", model.metrics_names)
    table.add_column("Global", np.round(score,4))

    # Cria uma tabela dos resultados por classe
    for i, s_class in enumerate(score_by_class):
        table.add_column(CLASSES[i], np.round(s_class,4))

    if VERBOSE['EVALUATION']:
        print()
        print(evaluation_name)
        print(table)

    all_scores = {
        'model_name': model.name,
        'name': evaluation_name,
        'loss': score,
        'loss_by_class': score_by_class,
        'confusion': cm.tolist(),
        'auc': list(roc_auc.values())
    }

    save_evaluation(model.name, evaluation_name, all_scores, table)

    return all_scores

In [10]:
def get_class_weight(y):
    """ Retorna o peso para cada classe
    
    Retorna um de peso que cada classe para que se mitigue o problema de desbalanceamento do conjunto de dados.

    Args:
        y: saída
        
    Returns:
        Peso para cada classe
    """

    class_weight = compute_class_weight(class_weight ='balanced', classes = CLASSES, y = y)
    return dict(zip(range(len(CLASSES)),class_weight))

In [11]:
def get_metrics():
    """ Retorna as métricas
    
    Retorna as métricas utilizadas para o problema.

    Args:
        Não há
        
    Returns:
        Array com as métricas
    """

    return [
        keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
        keras.metrics.AUC(name='AUC'),
        keras.metrics.AUC(name='PRC', curve='PR'), # precision-recall curve
    ]

In [12]:
def plot_execution_history(history, model):
    """ Desenha o histórico do treinamento
    
    Desenha uma curva para cada métrica, incluindo a função de perda, pelas épocas.

    Args:
        history: histórico das métricas
        model: modelo de classificação
        
    Returns:
        Não há
    """

    result_directory = os.path.join(RESULT_PATH, model.name)
    image_path = os.path.join(result_directory,'execution.png')

    # Se houver um conjunto dados de validação, o histórico de métricas possuirá o dobro de curvas
    if VALIDATION_PERCENTAGE > 0:
        nplots = len(history.values())/2
    else:
        nplots = len(history.values())
    fig = plt.figure(figsize=(16, nplots*16))
    # Apresentando os gráficos em somente 1 coluna
    gs = fig.add_gridspec(nplots, 1)
    axs = gs.subplots(sharex=False, sharey=False)

    # Para cada métrica armazenada em history, desenhar sua curva conforme as épocas
    for i,h in enumerate(history.values()):
        if VALIDATION_PERCENTAGE > 0:
            axs[i].plot(h, label = 'Training' if int(i/nplots)==0 else 'Validation')
        else:
            axs[i].plot(h,label = 'Training')

    # Colocar uma legenda
    for i,n in enumerate(model.metrics_names):
        axs[i].set_xlabel('Epoch')
        axs[i].set_ylabel(n)
        axs[i].legend()
      
    plt.savefig(image_path)

    # Exibe a curva de treinamento
    if VERBOSE['EXECUTION']:
        plt.show()
    else:
        plt.close()
        
def run_model(model, training_param, x_train, y_train, x_val, y_val, class_weight):
    """ Desenha o histórico de execução do treinamento
    
    Desenha uma curva para cada métrica, incluindo a função de perda, pelas épocas.

    Args:
        model: modelo de classificação
        training_param: função que retorna parâmetros de treinamento
        x_train: entrada de treinamento
        y_train: saída de treinamento
        x_test: entrada de teste
        y_test: saída de teste
        class_weight: peso para cada classe
        
    Returns:
        Modelo de classificação,
        Histórico do treinamento
    """

    optimizer, batch_size, epochs = training_param()

    if VERBOSE['EXECUTION']:
        model.summary()

    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=get_metrics())

    if x_val is None:
        xy_val = None
    else:
        xy_val = (x_val, y_val)

    start_time = time.time()
    # É dado o peso para o treinamento de cada classe a fim de mitigar o problema do desbalanceamento
    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data= xy_val,
                    class_weight = class_weight,
                    verbose=1 if VERBOSE['EXECUTION'] else 0)

    elapsed_minute = round((time.time() - start_time)/60)

    save_model(model,training_param, history, elapsed_minute)
    plot_execution_history(history.history, model)
      
    return model, history

In [13]:
#############################################################################################################################################

In [14]:
def train_and_evaluate(models_func,training_param):
    """ Treina e avalia um array de modelos
    
    Faz o carregamento do conjunto de dados, faz o treinamento e a avaliação de cada um dos modelos.

    Args:
        models_func: array de modelos de classificação
        training_param: função que retorna parâmetros de treinamento
        
    Returns:
        Não há
    """

    if os.path.isdir(RESULT_PATH):
        print("Folder already exists")
        return

    input_shape, _ = analyse_dataset(TRAIN_DIR_PATH)
    _, _ = analyse_dataset(TEST_DIR_PATH)
    x_trainval, y_trainval = load_dataset(TRAIN_DIR_PATH)
    x_test, y_test = load_dataset(TEST_DIR_PATH)
    x_train, x_val, y_train, y_val = split_dataset(x_trainval, y_trainval)
    x_train_prepared , y_train_prepared, _ = prepare_dataset(x_train , y_train, input_shape)
    x_val_prepared , y_val_prepared, _ = prepare_dataset(x_val , y_val, input_shape)
    x_test_prepared , y_test_prepared, _ = prepare_dataset(x_test , y_test, input_shape)    
    for model_func in models_func:
        model = model_func(input_shape,len(CLASSES))
        print("Training:", model.name)
        model, history = run_model(model,training_param, x_train_prepared, y_train_prepared, x_val_prepared, y_val_prepared, get_class_weight(y_train))
        history = history.history
        evaluate_model('training',model, x_train_prepared, y_train_prepared)
        evaluate_model('validation', model, x_val_prepared, y_val_prepared)
        evaluate_model('test', model, x_test_prepared, y_test_prepared)
        print("done\n")

def reevaluate():
    """ Reavalia os modelos treinados
    
    Faz a reavaliação dos modelos já treinados.

    Args:
        Não há
        
    Returns:
        Não há
    """
    
    input_shape, _ = analyse_dataset()
    x_trainval, y_trainval = load_dataset(TRAIN_DIR_PATH)
    x_test, y_test = load_dataset(TEST_DIR_PATH)
    x_train, x_val, y_train, y_val = split_dataset(x_trainval, y_trainval)
    x_train_prepared , y_train_prepared, _ = prepare_dataset(x_train , y_train , input_shape)
    x_val_prepared , y_val_prepared, _ = prepare_dataset(x_val , y_val, input_shape)
    x_test_prepared , y_test_prepared, _ = prepare_dataset(x_test , y_test, input_shape)
    for f in os.listdir(RESULT_PATH): 
        model, _ = load_model(f)
        print("Reevaluating:", model.name)
        evaluate_model('training',model, x_train_prepared, y_train_prepared)
        evaluate_model('validation', model, x_val_prepared, y_val_prepared)
        evaluate_model('test', model, x_test_prepared, y_test_prepared)
        print("done\n")

In [3]:
def load_dataset_reshuffle():
    """ Carrega, mistura e reparticiona os conjuntos de dados de treinamento e teste

    Carrega e retorna as entradas e saídas do conjunto de dados de treinamento de teste, após serem carregados, embaralhados e reparticionados.

    Args:
        Não há
        
    Returns:
        Entrada do conjunto de dados de treinamento,
        Saída do conjunto de dados de treinamento,
        Entrada do conjunto de dados de teste,
        Saída do conjunto de dados de teste,
    """

    img_array_train = []
    class_array_train = []
    img_array_test = []
    class_array_test = []

    for c in CLASSES:
        class_path_train = os.path.join(TRAIN_DIR_PATH, c)
        class_path_test = os.path.join(TEST_DIR_PATH, c)
        
        imgs_path = []
        for img_name in os.listdir(class_path_train):
            imgs_path.append(os.path.join(class_path_train, img_name))

        for img_name in os.listdir(class_path_test):
            imgs_path.append(os.path.join(class_path_test, img_name)) 

        random.shuffle(imgs_path)
        
        # Porcentagem original de treino e teste: 80% - 20%
        test_size = int(len(imgs_path)*0.2)
        train_size = len(imgs_path) - test_size
            
        for i in range(train_size):
            img_array_train.append(image.imread(imgs_path[i]))
            class_array_train.append(c)

        for i in range(test_size):
            img_array_test.append(image.imread(imgs_path[train_size+i]))
            class_array_test.append(c)

    if VERBOSE['LOADING']:
        print("Loaded %d train images" % len(img_array_train))
        print("Loaded %d test images" % len(img_array_test))

    return np.array(img_array_train), np.array(class_array_train),np.array(img_array_test), np.array(class_array_test)

def train_and_evaluate_reshuffled_dataset(models_func,training_param):
    input_shape, _ = analyse_dataset(TRAIN_DIR_PATH)
    _, _ = analyse_dataset(TEST_DIR_PATH)
    x_trainval, y_trainval, x_test, y_test = load_dataset_reshuffle()

    x_train, x_val, y_train, y_val = split_dataset(x_trainval, y_trainval)
    x_train_prepared , y_train_prepared, _ = prepare_dataset(x_train , y_train , input_shape)
    x_val_prepared , y_val_prepared, _ = prepare_dataset(x_val , y_val, input_shape)
    x_test_prepared , y_test_prepared, _ = prepare_dataset(x_test , y_test, input_shape)
    for model_func in models_func:
        model = model_func(input_shape,len(CLASSES))
        print("Training:", model.name)
        model, history = run_model(model,training_param, x_train_prepared, y_train_prepared, x_val_prepared, y_val_prepared, get_class_weight(y_train))
        history = history.history
        evaluate_model('training',model, x_train_prepared, y_train_prepared)
        evaluate_model('validation', model, x_val_prepared, y_val_prepared)
        evaluate_model('test', model, x_test_prepared, y_test_prepared)
        print("done\n")

